##Week 4 Logistics Regression

### Import Libraries

In [1]:
import numpy as np
print "numpy version: ", np.__version__
import sys
print sys.version
import pylab
%pylab
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 
import seaborn
import scipy

%matplotlib inline

# bug fix for display formats to avoid run time errors
pd.set_option('display.float_format', lambda x:'%.2f'%x)

# turn off warnings printing to screen
import warnings
warnings.filterwarnings('ignore')

numpy version:  1.9.2
2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 16:44:52) [MSC v.1500 64 bit (AMD64)]
Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


###Load the Data

In [2]:
data = pd.read_csv('D:/Data/CourseraClass/gapminder.csv')
print ("Number of records loaded: "), len(data)

Number of records loaded:  213


###Data Management

Subset the data to remove missing values from the variables of interest.

In [3]:
data['suicideper100th'] = data['suicideper100th'].replace(r'\s+', np.nan, regex=True)
data['internetuserate'] = data['internetuserate'].replace(r'\s+', np.nan, regex=True)
data['incomeperperson']= data['incomeperperson'].replace(r'\s+', np.nan, regex=True)
data['urbanrate']= data['urbanrate'].replace(r'\s+', np.nan, regex=True)
sub = data.dropna(subset = ['suicideper100th', 'internetuserate', 'incomeperperson', 'urbanrate'])
sub1 = sub[['suicideper100th', 'internetuserate', 'incomeperperson', 'urbanrate']]
print (sub1.head())
print (sub1.tail())
print ("Number of records remaining: "), len(sub1)

    suicideper100th   internetuserate   incomeperperson urbanrate
1  7.69932985305786  44.9899469578783  1914.99655094922     46.72
2   4.8487696647644  12.5000733055148  2231.99333515006     65.22
3  5.36217880249023                81  21943.3398976022     88.92
4  14.5546770095825  9.99995388324075  1381.00426770244      56.7
5  2.16184329986572  80.6454545454545  11894.4640745081     30.46
      suicideper100th   internetuserate   incomeperperson urbanrate
207           4.11962  35.8504369560241  5528.36311387522     93.32
208  11.6533222198486  27.8518215557703  722.807558834445     27.84
210  6.26578903198242  12.3497504635596    610.3573673206     30.64
211  12.0190362930298   10.124986462443  432.226336974583     35.42
212  13.9052667617798  11.5004153209768  320.771889948584     37.34
Number of records remaining:  173


Convert fields to numeric

In [4]:
sub1['suicideper100th'] = sub1['suicideper100th'].convert_objects(convert_numeric=True)
sub1['internetuserate'] = sub1['internetuserate'].convert_objects(convert_numeric=True)
sub1['incomeperperson'] = sub1['incomeperperson'].convert_objects(convert_numeric=True)
sub1['urbanrate'] = sub1['urbanrate'].convert_objects(convert_numeric=True)

Create categorial response based on percentiles. Those in the lower 50 percentile will be coded 0 to indicate lack of a high rate. Those in the upper 50 percentile will be coded as 1 to indicate presence of a high rate.

In [5]:
sub1['suiciderate'] = 0
sub1['suiciderate'][(sub1['suicideper100th']>=8.3)] = 1
print ("The number of records in the upper 50 percentile is: "), sub1['suiciderate'].sum()

The number of records in the upper 50 percentile is:  86


###Numerical Summaries of the Data

In [6]:
print ("The minimum value for Internet use rate is: "), round(sub1["internetuserate"].min(),3)
print ("The maximum value for Internet use rate is: "), round(sub1["internetuserate"].max(),3)
print ("The median value for Internet use rate is: "), round(sub1["internetuserate"].median(),3)
print ("The mean value for Internet use rate is: "), round(sub1["internetuserate"].mean(),3)
print ("The standard deviation for Internet use rate is: "), round(sub1["internetuserate"].std(),3)
print
print ("The minimum value for Income per Person is: "), round(sub1["incomeperperson"].min(),3)
print ("The maximum value for Income per Person is: "), round(sub1["incomeperperson"].max(),3)
print ("The median value for Income per Person is: "), round(sub1["incomeperperson"].median(),3)
print ("The mean value for Income per Person is: "), round(sub1["incomeperperson"].mean(),3)
print ("The standard deviation for Income per Person is: "), round(sub1["incomeperperson"].std(),3)
print
print ("The minimum value for Urban rate is: "), round(sub1["urbanrate"].min(),3)
print ("The maximum value for Urban rate is: "), round(sub1["urbanrate"].max(),3)
print ("The median value for Urban rate is: "), round(sub1["urbanrate"].median(),3)
print ("The mean value for Urban rate is: "), round(sub1["urbanrate"].mean(),3)
print ("The standard deviation for Urban rate is: "), round(sub1["urbanrate"].std(),3)
print
print ("The minimum value for Suicide rate is: "), round(sub1["suicideper100th"].min(),3)
print ("The maximum value for Suicide rate is: "), round(sub1["suicideper100th"].max(),3)
print ("The median value for Suicide rate is: "), round(sub1["suicideper100th"].median(),3)
print ("The mean value for Suicide rate is: "), round(sub1["suicideper100th"].mean(),3)
print ("The standard deviation for Suicide rate is: "), round(sub1["suicideper100th"].std(),3)

The minimum value for Internet use rate is:  0.21
The maximum value for Internet use rate is:  95.638
The median value for Internet use rate is:  28.999
The mean value for Internet use rate is:  34.018
The standard deviation for Internet use rate is:  27.737

The minimum value for Income per Person is:  103.776
The maximum value for Income per Person is:  52301.587
The median value for Income per Person is:  2481.719
The mean value for Income per Person is:  7196.814
The standard deviation for Income per Person is:  10250.664

The minimum value for Urban rate is:  10.4
The maximum value for Urban rate is:  100.0
The median value for Urban rate is:  56.76
The mean value for Urban rate is:  55.148
The standard deviation for Urban rate is:  22.89

The minimum value for Suicide rate is:  0.201
The maximum value for Suicide rate is:  35.753
The median value for Suicide rate is:  8.283
The mean value for Suicide rate is:  9.721
The standard deviation for Suicide rate is:  6.322


##Regression Analysis

###Center the Quantitative Variables

In [17]:
# center quantitative IVs for regression analysis
sub1['internetuserate_c'] = (sub1['internetuserate'] - sub1['internetuserate'].mean())
print (sub1['internetuserate_c'].mean()) 
sub1['urbanrate_c'] = (sub1['urbanrate'] - sub1['urbanrate'].mean())
print (sub1['urbanrate_c'].mean()) 
sub1['incomeperperson_c'] = (sub1['incomeperperson'] - sub1['incomeperperson'].mean())
print (sub1['incomeperperson_c'].mean()) 

-6.69470901323e-15
2.75181290728e-15
-1.01463859793e-12


#### Logistic Regression using Internet Use Rate

In [23]:
lreg1 = smf.logit(formula = 'suiciderate ~ internetuserate', data = sub1).fit()
print (lreg1.summary())
# odds ratios
print ("Odds Ratios")
print (numpy.exp(lreg1.params))

# odd ratios with 95% confidence intervals
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (numpy.exp(conf))

Optimization terminated successfully.
         Current function value: 0.684935
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:            suiciderate   No. Observations:                  173
Model:                          Logit   Df Residuals:                      171
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Dec 2015   Pseudo R-squ.:                 0.01182
Time:                        17:53:07   Log-Likelihood:                -118.49
converged:                       True   LL-Null:                       -119.91
                                        LLR p-value:                   0.09219
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept          -0.3285      0.243     -1.350      0.177        -0.805     0.148
internetuserat

##### Interpret Results
The results of the linear regression model indicated that Income per person (Beta=-6.732e-05, p=.393) was not significantly associated with suicide rates.  The results of the linear regression model indicated that Internet Usage Rate (Beta=0.0724, p=0.022) and Urban Rate (Beta=,-0.0686 p=.014) were significanly associated with suicide rates.  Internet usage was positively associated with suicide rates while urban rate was negatively associated with suicide rates.  


In [21]:
lreg1 = smf.logit(formula = 'suiciderate ~ urbanrate', data = sub1).fit()
print (lreg1.summary())
# odds ratios
print ("Odds Ratios")
print (numpy.exp(lreg1.params))

# odd ratios with 95% confidence intervals
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (numpy.exp(conf))

Optimization terminated successfully.
         Current function value: 0.688831
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:            suiciderate   No. Observations:                  173
Model:                          Logit   Df Residuals:                      171
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Dec 2015   Pseudo R-squ.:                0.006203
Time:                        17:52:43   Log-Likelihood:                -119.17
converged:                       True   LL-Null:                       -119.91
                                        LLR p-value:                    0.2226
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.4384      0.401      1.094      0.274        -0.347     1.224
urbanrate     -0.0082      0.

In [22]:
lreg1 = smf.logit(formula = 'suiciderate ~ incomeperperson', data = sub1).fit()
print (lreg1.summary())
# odds ratios
print ("Odds Ratios")
print (numpy.exp(lreg1.params))

# odd ratios with 95% confidence intervals
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (numpy.exp(conf))

Optimization terminated successfully.
         Current function value: 0.681824
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:            suiciderate   No. Observations:                  173
Model:                          Logit   Df Residuals:                      171
Method:                           MLE   Df Model:                            1
Date:                Tue, 08 Dec 2015   Pseudo R-squ.:                 0.01631
Time:                        17:52:48   Log-Likelihood:                -117.96
converged:                       True   LL-Null:                       -119.91
                                        LLR p-value:                   0.04794
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept          -0.2265      0.188     -1.205      0.228        -0.595     0.142
incomeperperso

In [24]:
lreg1 = smf.logit(formula = 'suiciderate ~ urbanrate+incomeperperson', data = sub1).fit()
print (lreg1.summary())
# odds ratios
print ("Odds Ratios")
print (numpy.exp(lreg1.params))

# odd ratios with 95% confidence intervals
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (numpy.exp(conf))

Optimization terminated successfully.
         Current function value: 0.656231
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            suiciderate   No. Observations:                  173
Model:                          Logit   Df Residuals:                      170
Method:                           MLE   Df Model:                            2
Date:                Tue, 08 Dec 2015   Pseudo R-squ.:                 0.05324
Time:                        17:54:12   Log-Likelihood:                -113.53
converged:                       True   LL-Null:                       -119.91
                                        LLR p-value:                  0.001689
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           0.9188      0.437      2.101      0.036         0.062     1.776
urbanrate     

In [26]:
lreg1 = smf.logit(formula = 'suiciderate ~ incomeperperson+urbanrate+internetuserate', data = sub1).fit()
print (lreg1.summary())
# odds ratios
print ("Odds Ratios")
print (numpy.exp(lreg1.params))

# odd ratios with 95% confidence intervals
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (numpy.exp(conf))

Optimization terminated successfully.
         Current function value: 0.650400
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            suiciderate   No. Observations:                  173
Model:                          Logit   Df Residuals:                      169
Method:                           MLE   Df Model:                            3
Date:                Tue, 08 Dec 2015   Pseudo R-squ.:                 0.06165
Time:                        17:55:53   Log-Likelihood:                -112.52
converged:                       True   LL-Null:                       -119.91
                                        LLR p-value:                  0.002010
                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           0.8954      0.441      2.029      0.042         0.030     1.760
incomeperperso